# Model Preparation

In [4]:
# Import necessary libraries
import keras
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

# Load pre-trained ResNet50 model with ImageNet weights
resnet50_model = ResNet50(weights='imagenet')

In [6]:
# Extract the dataset from the files
train_path  = r"C:\Users\krise\FYP\dataset\Train"
valid_path  = r"C:\Users\krise\FYP\dataset\Validation"
test_path   = r"C:\Users\krise\FYP\dataset\Test"

In [7]:
# Categorize the image
class_type = {0:'Earth', 1:'Fire', 2:'Moon', 3:'Soil', 4:'Water'}

## Checking the image and resize

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
target_size = (224, 224) # Adjusted the image size

train = train_datagen.flow_from_directory(
    train_path,
    target_size=target_size,)

Found 4299 images belonging to 5 classes.


In [9]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
target_size = (224, 224) # Adjusted the image size

test = test_datagen.flow_from_directory(
    test_path,
    target_size=target_size,)

Found 4311 images belonging to 5 classes.


In [10]:
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
target_size = (224, 224) # Adjusted the image size

valid = valid_datagen.flow_from_directory(
    valid_path,
    target_size=target_size,)

Found 4320 images belonging to 5 classes.


In [10]:
# Implement the modelling "ResNet50"
model = ResNet50( include_top=False, input_shape=(224, 224, 3))

In [11]:
for layer in model.layers:
    layer.trainable = False

In [12]:
# Import necessary libraries
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Extracting the output from the model
x = model.output

# Flatten the output and add a dense layer with 1024 units and ReLU activation
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x)

# Add a dropout layer with a rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add the final dense layer with 5 units and softmax activation for predictions
predictions = keras.layers.Dense(5, activation='softmax')(x)

# Create a new model with the specified input and the prediction layer
model = keras.models.Model(inputs=model.input, outputs=predictions)

# Compile the model with Adam optimizer, categorical crossentropy loss, and accuracy metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to monitor validation accuracy and stop training if no improvement
es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
# Model checkpoint to save the best model based on validation accuracy
mc = ModelCheckpoint(filepath="ResNet50.h5", monitor="val_accuracy", verbose=1, save_best_only= True)

In [11]:
# Train the data
history = model.fit(
    train,
    steps_per_epoch=10, # Adjusted how many epoch
    validation_data = valid,
    validation_steps=32,
    callbacks=[mc],
    epochs=10) # Adjusted how many step to run

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 43.6867 - accuracy: 0.4000
Epoch 1: val_accuracy improved from -inf to 0.58301, saving model to ResNet50.h5


C:\Users\krise\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 62s 6s/step - loss: 43.6867 - accuracy: 0.4000 - val_loss: 18.8026 - val_accuracy: 0.5830
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 12.7145 - accuracy: 0.7156
Epoch 2: val_accuracy improved from 0.58301 to 0.73145, saving model to ResNet50.h5
10/10 [==============================] - 59s 6s/step - loss: 12.7145 - accuracy: 0.7156 - val_loss: 12.0691 - val_accuracy: 0.7314
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 8.8935 - accuracy: 0.7906
Epoch 3: val_accuracy improved from 0.73145 to 0.73828, saving model to ResNet50.h5
10/10 [==============================] - 59s 6s/step - loss: 8.8935 - accuracy: 0.7906 - val_loss: 11.5323 - val_accuracy: 0.7383
Epoch 4/10
10/10 [==============================] - ETA: 0s - loss: 8.8863 - accuracy: 0.8000
Epoch 4: val_accuracy did not improve from 0.73828
10/10 [==============================] - 56s 6s/step - loss: 8.8863 - accuracy: 0.8000 - val_loss: 13.3860

# Model Use

In [1]:
history.history

# Plotting the loss
fig = plt.figure()
plt.plot(history.history['loss'], color='teal',label='loss')
plt.plot(history.history['val_loss'], color='orange',label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc= "upper left")
plt.show

# Plotting the accuracy
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal',label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange',label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc= "upper left")
plt.show

NameError: name 'history' is not defined

In [14]:
from keras.models import load_model
model = load_model("ResNet50.h5")

In [ ]:
# Evaluate the model using the test generator
evaluation = model.evaluate(test)

# Print the evaluation results
print(f"The accuracy of your model is = {evaluation[1] * 100:.2f}%")

  9/135 [=>............................] - ETA: 2:18 - loss: 0.0904 - accuracy: 0.9931

In [15]:
import tensorflow as tf
import keras.utils as image # Importing image utilities from Keras

# Function to load and preprocess an image
def get_img_array(img_path):
    path = img_path
    img = tf.keras.utils.load_img(path, target_size=(224,224,3))
    img = image.img_to_array(img)
    img = np.expand_dims(img , axis= 0 )
    return img

In [17]:
path = r'C:\Users\krise\FYP\google_images\Water.png'

img = get_img_array(path)

In [18]:
predicted_class  = class_type[np.argmax(model.predict(img))] 
#class_labels = train_generator.class_indices
#class_name = list(class_labels.keys())[list(class_labels.values()).index(predicted_class)]
print("The image belongs to the class:", predicted_class)

1/1 [==============================] - 1s 785ms/step
The image belongs to the class: Water
